In [1]:
#Set 2

In [2]:
#P9

In [6]:
block = b"YELLOW SUBMARINE"

In [13]:
padding = (20 - len(block)%20)*'\x04'

In [16]:
block + padding

'YELLOW SUBMARINE\x04\x04\x04\x04'

In [17]:
#P10

In [130]:
import urllib2
from Crypto.Cipher import AES
import sys
sys.path.append( 'Set1' )
from P5 import repkeyxor

In [131]:
url = 'http://cryptopals.com/static/challenge-data/10.txt'

In [132]:
text = urllib2.urlopen(url).read()

In [133]:
text = ''.join(text.split()).decode('base64')

In [134]:
key = "YELLOW SUBMARINE"
iv = '\x00'*16

In [135]:
cipher = AES.new(key, AES.MODE_ECB)

In [136]:
plaintext = ""
plaintext += repkeyxor(cipher.decrypt(text[:16]), iv).decode('hex')
for i in range(1,len(text)/16):
    plaintext += repkeyxor(cipher.decrypt(text[16*i:16*i + 16]), text[16*i - 16:16*i]).decode('hex')

In [137]:
#range(1,len(text)/16)
print plaintext

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 
